In [1]:
!pip install scipy==1.7.3
!pip install numpy==1.23.5
!pip install pandas==1.5.3
!pip install lightgbm==3.3.2
!pip install catboost==1.2
!pip install xgboost==1.7.6
!pip install sklearn==1.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 73.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.1
    Uninstalling scipy-1.11.1:
      Successfully uninstalled scipy-1.11.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
chex 0.1.81 requires numpy>=1.25.0, but you have numpy 1.22.4 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
pointpats 2.3.0 requires scipy>=1.8.0, but you have scipy 1.7.3 which is incompatible.
py

In [2]:
import scipy
import numpy
import pandas
import lightgbm
import catboost
import xgboost
import sklearn
print(scipy.__version__)
print(numpy.__version__)
print(pandas.__version__)
print(lightgbm.__version__)
print(catboost.__version__)
print(xgboost.__version__)
print(sklearn.__version__)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


1.7.3
1.23.5
1.5.3
3.3.2
1.2
1.7.6
1.2.2


In [3]:
import os
import warnings
import statistics
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import lightgbm as lgbm
from scipy import stats
from scipy.stats import hmean,gmean
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import HistGradientBoostingClassifier,RandomForestClassifier,BaggingClassifier,ExtraTreesClassifier
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GroupKFold
pd.set_option('display.max_columns', 500)
warnings.filterwarnings("ignore")

In [4]:
version_nb = 'v5_kaggle'

In [5]:
df_meta_train = pd.read_csv('/kaggle/input/fia-zindi-kp-data-creation/train.csv',parse_dates=['endTime'])
df_meta_test = pd.read_csv('/kaggle/input/fia-zindi-kp-data-creation/test.csv',parse_dates=['endTime'])
df_ss = pd.read_csv('/kaggle/input/fia-zindi-kp-data-creation/sample_submission.csv')
df_meta_test.sort_values(['ID','endTime'],ascending=True,ignore_index=True,inplace=True)
df_meta_train.sort_values(['ID','endTime'],ascending=True,ignore_index=True,inplace=True)
print(df_meta_train.shape, df_meta_test.shape, df_ss.shape)
display(df_meta_train.head(1))
display(df_meta_test.head(1))
display(df_ss.head(1))

(908922, 12) (175150, 12) (1932, 2)


,ID,NE ID,endTime,access_success_rate,resource_utilition_rate,TA,bler,cqi,mcs,data_rate,fault_duration,relation
0,B0001-12_1,B0001-12,2023-02-09 01:00:00,99.519744,29.417,0.636289,10.12688,8.640631,12.523579,32.209893,0.0,0.0


,ID,NE ID,endTime,access_success_rate,resource_utilition_rate,TA,bler,cqi,mcs,data_rate,fault_duration,relation
0,B0017-25_24,B0017-25,2023-02-20 06:00:00,99.714558,8.678,3.71541,7.653169,5.964988,5.403557,6.815789,0.0,0.0


,ID,data_rate_t+1_trend
0,B0017-25_24,0


In [6]:
df_meta_train_features = df_meta_train[df_meta_train['fault_duration']==0].reset_index(drop=True)
df_meta_test_features = df_meta_test[df_meta_test['fault_duration']==0].reset_index(drop=True)
print(df_meta_train_features.shape,df_meta_test_features.shape)
df_train = df_meta_train[df_meta_train['fault_duration']>0].reset_index(drop=True)
df1 = df_meta_train_features.groupby(['ID'])[['data_rate']].last().add_suffix("_t-1").reset_index()
df_train = df_train.merge(df1,on=['ID'],how='left')
df_train['data_rate_t+1_trend'] = np.where(df_train['data_rate']<df_train['data_rate_t-1'],1,0)


# df_train_first = df_train.groupby(['ID']).first().reset_index()
# df_train_after_first = df_train.groupby(["ID"]).apply(lambda group: group.iloc[1:]).reset_index(drop=True)
for col in ['access_success_rate','resource_utilition_rate', 'TA', 'bler', 'cqi', 'mcs']:
    del df_train[col]
# df_train = pd.concat([df_train_first,df_train_after_first], ignore_index=True)

print(df_train.groupby(['ID']).first()['data_rate_t+1_trend'].value_counts(normalize=True))
print(df_train['data_rate_t+1_trend'].value_counts(normalize=True))
df_train.rename(columns={'data_rate':'TARGET_data_rate','data_rate_t+1_trend':'TARGET_data_rate_t+1_trend'},inplace=True)
df_train = df_train[['ID', 'NE ID', 'endTime',  'fault_duration','relation', 'data_rate_t-1',
                     'TARGET_data_rate', 'TARGET_data_rate_t+1_trend']].copy()
display(df_train)
df_test = df_meta_test[df_meta_test['fault_duration']>0].reset_index(drop=True)
df1 = df_meta_test_features.groupby(['ID'])[['data_rate']].last().add_suffix("_t-1").reset_index()
df_test = df_test.merge(df1,on=['ID'],how='left')
for col in ['access_success_rate','resource_utilition_rate', 'TA', 'bler', 'cqi', 'mcs','data_rate']:
    del df_test[col]

display(df_test)
df_meta_train_features['data'] = 'no_fault'
df_meta_test_features['data'] = 'no_fault'
df_train['data'] = 'train_fault'
df_test['data'] = 'test_fault'
df_combined = pd.concat([df_meta_train_features,df_train,df_meta_test_features,df_test]).reset_index(drop=True)

(813138, 12) (173218, 12)
0    0.508407
1    0.491593
Name: data_rate_t+1_trend, dtype: float64
1    0.526936
0    0.473064
Name: data_rate_t+1_trend, dtype: float64


,ID,NE ID,endTime,fault_duration,relation,data_rate_t-1,TARGET_data_rate,TARGET_data_rate_t+1_trend
0,B0001-12_1,B0001-12,2023-02-20 13:00:00,91.0,0.942467,23.253083,23.910174,0
1,B0001-13_1,B0001-13,2023-02-20 13:00:00,91.0,0.942467,22.537942,28.421796,0
2,B0001-15_1,B0001-15,2023-02-20 13:00:00,91.0,0.942467,5.711927,19.964336,0
3,B0001-22_1,B0001-22,2023-02-20 13:00:00,94.0,0.946876,26.605883,13.039390,1
4,B0001-23_1,B0001-23,2023-02-20 13:00:00,94.0,0.946876,24.037401,35.187848,0
...,...,...,...,...,...,...,...,...
95779,B0729-13_1,B0729-13,2023-02-28 02:00:00,209.0,0.990000,21.438768,21.961071,0
95780,B0729-15_1,B0729-15,2023-02-28 01:00:00,950.0,0.990000,4.248081,4.510619,0
95781,B0729-15_1,B0729-15,2023-02-28 02:00:00,209.0,0.990000,4.248081,6.097206,0
95782,B0729-22_1,B0729-22,2023-02-28 01:00:00,950.0,1.000000,14.851790,20.272872,0


,ID,NE ID,endTime,fault_duration,relation,data_rate_t-1
0,B0017-25_24,B0017-25,2023-02-20 09:00:00,301.0,0.654162,1.175289
1,B0017-25_25,B0017-25,2023-02-20 21:00:00,145.0,0.654162,0.966620
2,B0017-25_26,B0017-25,2023-02-21 00:00:00,250.0,0.654162,1.561278
3,B0017-25_27,B0017-25,2023-02-21 06:00:00,1971.0,0.654162,10.963935
4,B0017-32_1,B0017-32,2023-02-14 21:00:00,3461.0,0.654162,4.471131
...,...,...,...,...,...,...
1927,B0122-43_1,B0122-43,2023-02-11 15:00:00,616.0,1.000000,16.973420
1928,B0122-43_2,B0122-43,2023-02-13 15:00:00,97.0,0.990000,8.685347
1929,B0122-45_1,B0122-45,2023-02-11 15:00:00,616.0,1.000000,3.946578
1930,B0122-45_2,B0122-45,2023-02-13 15:00:00,97.0,0.990000,2.349000


In [7]:
%%time
df_total = df_combined.sort_values(['NE ID','endTime'],ascending=True,ignore_index=True)
del df_total['data_rate_t-1']
df_total_resample = df_total.set_index('endTime').groupby('NE ID').resample('H').asfreq().drop(['NE ID'], 1).reset_index()
df_total_resample = df_total_resample.sort_values(['NE ID','endTime'],ascending=True,ignore_index=True)
print(df_total.shape, df_total_resample.shape)

(1084072, 15) (1093242, 15)
CPU times: user 5.74 s, sys: 243 ms, total: 5.99 s
Wall time: 5.99 s


In [8]:
# df_total['access_success_rate_by_resource_utilition_rate'] = df_total['access_success_rate'] / df_total['resource_utilition_rate']
# df_total_resample['access_success_rate_by_resource_utilition_rate'] = df_total_resample['access_success_rate'] / df_total_resample['resource_utilition_rate']


In [9]:
# df_total = df_total[df_total['data_rate']!=0].reset_index(drop=True)
all_shits = [1,2,3] #list(np.arange(1,49)) # 
for shift_i in tqdm(all_shits):
    for col in ['endTime','access_success_rate','resource_utilition_rate','TA','bler','cqi','mcs','data_rate',
               'fault_duration','relation']:
        df_total[f'{col}_T-{shift_i}'] = df_total.groupby(['NE ID'])[col].shift(shift_i)
    
for shift_i in tqdm(all_shits):
    for col in ['endTime','access_success_rate','resource_utilition_rate','TA','bler','cqi','mcs','data_rate',
               'fault_duration','relation']:
        df_total[f'{col}_T+{shift_i}'] = df_total.groupby(['NE ID'])[col].shift(-shift_i)
        

for shift_i in tqdm(all_shits):
    df_total[f'endTime_T+{shift_i}_hours_elapsed'] = (df_total[f'endTime_T+{shift_i}']-df_total['endTime']).dt.total_seconds() / 3600
    df_total[f'endTime_T-{shift_i}_hours_elapsed'] = (df_total[f'endTime_T-{shift_i}']-df_total['endTime']).dt.total_seconds() / 3600
    del df_total[f'endTime_T+{shift_i}']
    del df_total[f'endTime_T-{shift_i}']
    

100%|██████████| 3/3 [00:00<00:00, 37.56it/s]


In [10]:
# df_total[f'data_rate_T-1-T-2'] = df_total.groupby(['NE ID'])['data_rate'].shift(1)- df_total.groupby(['NE ID'])['data_rate'].shift(2)
# df_total[f'data_rate_T-2-T-3'] = df_total.groupby(['NE ID'])['data_rate'].shift(2)- df_total.groupby(['NE ID'])['data_rate'].shift(3)
# df_total[f'data_rate_T-3-T-4'] = df_total.groupby(['NE ID'])['data_rate'].shift(3)- df_total.groupby(['NE ID'])['data_rate'].shift(4)


In [11]:
for shift_i in tqdm([24,48,72]):
    for col in ['access_success_rate','resource_utilition_rate','TA','bler',
                'cqi','mcs','data_rate','fault_duration','relation']:
        df_total_resample[f'{col}_T-{shift_i}'] = df_total_resample.groupby(['NE ID'])[col].shift(shift_i)

for shift_i in tqdm([24,48,72]):
    for col in ['access_success_rate','resource_utilition_rate','TA','bler',
                'cqi','mcs','data_rate','fault_duration','relation']:
        df_total_resample[f'{col}_T+{shift_i}'] = df_total_resample.groupby(['NE ID'])[col].shift(-shift_i)


100%|██████████| 3/3 [00:02<00:00,  1.30it/s]


In [12]:
df_tot_fault_et = df_total_resample[df_total_resample['fault_duration']>0].groupby(['NE ID']).agg({'endTime':lambda x: list(x)}).add_prefix("fault_").reset_index()


In [13]:
# df_total_resample[f'data_rate_T-24-T-25'] = df_total_resample.groupby(['NE ID'])['data_rate'].shift(24)- df_total_resample.groupby(['NE ID'])['data_rate'].shift(25)
# df_total_resample[f'data_rate_T-48-T-49'] = df_total_resample.groupby(['NE ID'])['data_rate'].shift(48)- df_total_resample.groupby(['NE ID'])['data_rate'].shift(49)
# df_total_resample[f'data_rate_T-72-T-73'] = df_total_resample.groupby(['NE ID'])['data_rate'].shift(72)- df_total_resample.groupby(['NE ID'])['data_rate'].shift(73)

In [14]:
df_total['month'] = df_total['endTime'].dt.month
df_total['day'] = df_total['endTime'].dt.day
df_total['hour'] = df_total['endTime'].dt.hour
df_total['weekday_number'] = df_total['endTime'].dt.weekday
df_total['hour_sin'] = np.sin(2 * np.pi * df_total['hour'] / 24)
df_total['hour_cos'] = np.cos(2 * np.pi * df_total['hour'] / 24)
# df_total['weekday_sin'] = np.sin(2 * np.pi * df_total['weekday_number'] / 7)
# df_total['weekday_cos'] = np.cos(2 * np.pi * df_total['weekday_number'] / 7)
# df_total['day_sin'] = np.sin(2 * np.pi * df_total['day'] / 31)
# df_total['day_cos'] = np.cos(2 * np.pi * df_total['day'] / 31)

# df_total = df_total.groupby(['ID']).first().reset_index()


In [15]:
# df_mean_drates = df_total.groupby(['NE ID','hour']).agg({'data_rate':['mean','std','min','max']})
# df_mean_drates.columns = [i[0]+"_"+i[1] for i in df_mean_drates]
# df_mean_drates = df_mean_drates.reset_index()
# df_total = df_total.merge(df_mean_drates,on=['NE ID','hour'],how='left')

In [16]:
category_cols = ['month','day','hour','weekday_number']

In [17]:
df_total['last_10_data_rates'] = np.vstack([np.array(df_total.groupby(['NE ID'])['data_rate'].shift(i).fillna(-1)) for i in range(1,11)]).T.tolist()


In [18]:
df_train = df_total[df_total['data']=='train_fault'].groupby(['ID']).first().reset_index()
df_train.drop(columns=['access_success_rate','resource_utilition_rate',
                      'TA','bler','cqi','mcs','data_rate','data','TARGET_data_rate'],
              inplace=True)
df_train = df_train.merge(df_total_resample[[col for col in df_total_resample if col not in ['ID','access_success_rate',
                                                 'resource_utilition_rate','TA','bler',
                                                 'cqi','mcs','data_rate','data',
                                                 'TARGET_data_rate','fault_duration',
                                                 'relation','TARGET_data_rate_t+1_trend']]],
              on=['NE ID','endTime'], how='left')

# #time since last fault
# df_train = df_train.merge(df_tot_fault_et,on='NE ID',how='left')
# df_train['fault_endTime'] = df_train.apply(lambda x: max([i for i in x['fault_endTime'] if i<x['endTime']]) if len([i for i in x['fault_endTime'] if i<x['endTime']])>0 else np.nan, axis=1)
# df_train['fault_endTime'] = (df_train['endTime']-df_train['fault_endTime']).dt.total_seconds() / 3600

print(df_train.shape)
display(df_train.head(1))

(7256, 127)


,ID,NE ID,endTime,fault_duration,relation,TARGET_data_rate_t+1_trend,access_success_rate_T-1,resource_utilition_rate_T-1,TA_T-1,bler_T-1,cqi_T-1,mcs_T-1,data_rate_T-1,fault_duration_T-1,relation_T-1,access_success_rate_T-2,resource_utilition_rate_T-2,TA_T-2,bler_T-2,cqi_T-2,mcs_T-2,data_rate_T-2,fault_duration_T-2,relation_T-2,access_success_rate_T-3,resource_utilition_rate_T-3,TA_T-3,bler_T-3,cqi_T-3,mcs_T-3,data_rate_T-3,fault_duration_T-3,relation_T-3,access_success_rate_T+1,resource_utilition_rate_T+1,TA_T+1,bler_T+1,cqi_T+1,mcs_T+1,data_rate_T+1,fault_duration_T+1,relation_T+1,access_success_rate_T+2,resource_utilition_rate_T+2,TA_T+2,bler_T+2,cqi_T+2,mcs_T+2,data_rate_T+2,fault_duration_T+2,relation_T+2,access_success_rate_T+3,resource_utilition_rate_T+3,TA_T+3,bler_T+3,cqi_T+3,mcs_T+3,data_rate_T+3,fault_duration_T+3,relation_T+3,endTime_T+1_hours_elapsed,endTime_T-1_hours_elapsed,endTime_T+2_hours_elapsed,endTime_T-2_hours_elapsed,endTime_T+3_hours_elapsed,endTime_T-3_hours_elapsed,month,day,hour,weekday_number,hour_sin,hour_cos,last_10_data_rates,access_success_rate_T-24,resource_utilition_rate_T-24,TA_T-24,bler_T-24,cqi_T-24,mcs_T-24,data_rate_T-24,fault_duration_T-24,relation_T-24,access_success_rate_T-48,resource_utilition_rate_T-48,TA_T-48,bler_T-48,cqi_T-48,mcs_T-48,data_rate_T-48,fault_duration_T-48,relation_T-48,access_success_rate_T-72,resource_utilition_rate_T-72,TA_T-72,bler_T-72,cqi_T-72,mcs_T-72,data_rate_T-72,fault_duration_T-72,relation_T-72,access_success_rate_T+24,resource_utilition_rate_T+24,TA_T+24,bler_T+24,cqi_T+24,mcs_T+24,data_rate_T+24,fault_duration_T+24,relation_T+24,access_success_rate_T+48,resource_utilition_rate_T+48,TA_T+48,bler_T+48,cqi_T+48,mcs_T+48,data_rate_T+48,fault_duration_T+48,relation_T+48,access_success_rate_T+72,resource_utilition_rate_T+72,TA_T+72,bler_T+72,cqi_T+72,mcs_T+72,data_rate_T+72,fault_duration_T+72,relation_T+72
0,B0001-12_1,B0001-12,2023-02-20 13:00:00,91.0,0.942467,0.0,100.0,29.382,0.763943,10.336147,8.241281,11.089316,23.253083,0.0,0.0,100.0,45.419,0.811247,10.261306,8.565688,12.344837,21.86461,0.0,0.0,100.0,45.113,0.75943,10.249399,8.600942,12.360695,24.524838,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,-2.0,NaN,-3.0,2,20,13,0,-0.258819,-0.965926,"[23.25308286587564, 21.864609567544047, 24.524...",100.0,25.604,0.891304,10.965842,8.320807,11.496593,17.156717,0.0,0.0,100.0,37.071,0.862605,11.659417,8.081238,10.35328,17.371207,0.0,0.0,99.951714,31.598,0.834253,10.487901,8.34011,9.946214,22.470461,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_test = df_total[df_total['data']=='test_fault'].reset_index(drop=True)
df_test.drop(columns=['access_success_rate','resource_utilition_rate',
                      'TA','bler','cqi','mcs','data_rate','data',
                      'TARGET_data_rate','TARGET_data_rate_t+1_trend'],
             inplace=True)
df_test = df_test.merge(df_total_resample[[col for col in df_total_resample if col not in ['ID','access_success_rate',
                                                 'resource_utilition_rate','TA','bler',
                                                 'cqi','mcs','data_rate','data',
                                                 'TARGET_data_rate','fault_duration',
                                                 'relation','TARGET_data_rate_t+1_trend']]],
              on=['NE ID','endTime'], how='left')

# #time since last fault
# df_test = df_test.merge(df_tot_fault_et,on='NE ID',how='left')
# df_test['fault_endTime'] = df_test.apply(lambda x: max([i for i in x['fault_endTime'] if i<x['endTime']]) if len([i for i in x['fault_endTime'] if i<x['endTime']])>0 else np.nan, axis=1)
# df_test['fault_endTime'] = (df_test['endTime']-df_test['fault_endTime']).dt.total_seconds() / 3600


print(df_test.shape)
display(df_test.head(1))

(1932, 126)


,ID,NE ID,endTime,fault_duration,relation,access_success_rate_T-1,resource_utilition_rate_T-1,TA_T-1,bler_T-1,cqi_T-1,mcs_T-1,data_rate_T-1,fault_duration_T-1,relation_T-1,access_success_rate_T-2,resource_utilition_rate_T-2,TA_T-2,bler_T-2,cqi_T-2,mcs_T-2,data_rate_T-2,fault_duration_T-2,relation_T-2,access_success_rate_T-3,resource_utilition_rate_T-3,TA_T-3,bler_T-3,cqi_T-3,mcs_T-3,data_rate_T-3,fault_duration_T-3,relation_T-3,access_success_rate_T+1,resource_utilition_rate_T+1,TA_T+1,bler_T+1,cqi_T+1,mcs_T+1,data_rate_T+1,fault_duration_T+1,relation_T+1,access_success_rate_T+2,resource_utilition_rate_T+2,TA_T+2,bler_T+2,cqi_T+2,mcs_T+2,data_rate_T+2,fault_duration_T+2,relation_T+2,access_success_rate_T+3,resource_utilition_rate_T+3,TA_T+3,bler_T+3,cqi_T+3,mcs_T+3,data_rate_T+3,fault_duration_T+3,relation_T+3,endTime_T+1_hours_elapsed,endTime_T-1_hours_elapsed,endTime_T+2_hours_elapsed,endTime_T-2_hours_elapsed,endTime_T+3_hours_elapsed,endTime_T-3_hours_elapsed,month,day,hour,weekday_number,hour_sin,hour_cos,last_10_data_rates,access_success_rate_T-24,resource_utilition_rate_T-24,TA_T-24,bler_T-24,cqi_T-24,mcs_T-24,data_rate_T-24,fault_duration_T-24,relation_T-24,access_success_rate_T-48,resource_utilition_rate_T-48,TA_T-48,bler_T-48,cqi_T-48,mcs_T-48,data_rate_T-48,fault_duration_T-48,relation_T-48,access_success_rate_T-72,resource_utilition_rate_T-72,TA_T-72,bler_T-72,cqi_T-72,mcs_T-72,data_rate_T-72,fault_duration_T-72,relation_T-72,access_success_rate_T+24,resource_utilition_rate_T+24,TA_T+24,bler_T+24,cqi_T+24,mcs_T+24,data_rate_T+24,fault_duration_T+24,relation_T+24,access_success_rate_T+48,resource_utilition_rate_T+48,TA_T+48,bler_T+48,cqi_T+48,mcs_T+48,data_rate_T+48,fault_duration_T+48,relation_T+48,access_success_rate_T+72,resource_utilition_rate_T+72,TA_T+72,bler_T+72,cqi_T+72,mcs_T+72,data_rate_T+72,fault_duration_T+72,relation_T+72
0,B0017-25_24,B0017-25,2023-02-20 09:00:00,301.0,0.654162,99.357688,84.004,2.923368,14.209819,5.582824,5.667775,1.175289,0.0,0.0,99.927484,24.264,3.145402,10.015796,6.141206,6.544645,6.777114,0.0,0.0,99.714558,8.678,3.71541,7.653169,5.964988,5.403557,6.815789,0.0,0.0,99.264875,93.086,2.732594,16.947693,5.475508,4.719393,0.52254,0.0,0.0,99.642289,92.242,2.877206,15.083843,5.628569,5.051611,0.96662,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,0.654162,10.0,-1.0,11.0,-2.0,12.0,-3.0,2,20,9,0,0.707107,-0.707107,"[1.17528900605546, 6.777114273500261, 6.815788...",99.814929,55.07,3.171086,12.769518,5.834403,6.893467,3.292267,0.0,0.0,99.754199,78.256,2.889007,13.597193,5.825114,6.012004,2.046892,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.0,0.654162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
remove_non_unique_cols = []
for col in df_train:
    if col not in ['TARGET_data_rate','TARGET_data_rate_t+1_trend','last_10_data_rates']:
        if df_train[col].nunique()<=1:
            remove_non_unique_cols.append(col)
            print(col,":",df_train[col].nunique())

for col in df_test:
    if col not in ['TARGET_data_rate','TARGET_data_rate_t+1_trend','last_10_data_rates']:
        if df_test[col].nunique()<=1:
            remove_non_unique_cols.append(col)
            print(col,":",df_test[col].nunique())
remove_non_unique_cols

fault_duration_T-1 : 1
relation_T-1 : 1
fault_duration_T+1 : 1
relation_T+1 : 1


['fault_duration_T-1', 'relation_T-1', 'fault_duration_T+1', 'relation_T+1']

In [21]:
feature_set = [col for col in df_test.columns if col not in ['ID','NE ID','endTime','last_10_data_rates']]
print(len(feature_set))
feature_set = [col for col in feature_set if col not in remove_non_unique_cols]
print(len(feature_set))
target_variable = 'TARGET_data_rate_t+1_trend'
id_variable = 'ID'

122
118


In [22]:
for col in df_test:
    if col not in ['TARGET_data_rate','TARGET_data_rate_t+1_trend']:
        if df_test[col].isnull().sum()/len(df_test)>=0.8:
            print(col,":",df_test[col].isnull().sum()/len(df_test))

In [23]:
train_x = df_train.reset_index(drop=True)
train_y = train_x[target_variable]
test_x = df_test.reset_index(drop=True)

del train_x[target_variable]
test_id = test_x[id_variable]
# train_x = train_x[feature_set].copy()
# test_x = test_x[feature_set].copy()

print(train_x.shape, train_y.shape, test_x.shape)

(7256, 126) (7256,) (1932, 126)


In [24]:
train_x[category_cols] = train_x[category_cols].astype('category')
test_x[category_cols] = test_x[category_cols].astype('category')

In [25]:
def evaluate_F1_lgb(truth, predictions):  
    predictions = np.where(predictions < f1_threshold, 0, 1)
    f1 = f1_score(truth, predictions)
    return ('F1 Score', f1, True)

def evaluate_F1_xgb(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = f1_score(y_true, np.round(y_pred))
    return 'f1_err', -err

NFOLDS = 10
sc_pos_wt = 1.5
f1_threshold=0.5
seeds_rs = [42] #,1818,181812,1995,1212

In [26]:
#0.7300
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
pred_test = []
pred_x = []
pred_y = []
j=0
df_train_error_analysis_lgb = pd.DataFrame()
feature_importance_df = pd.DataFrame()
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('● Fold :', fold+1)
    pred_fold = []
    pred_fold_test = []
    for rs_ in seeds_rs:
        model = LGBMClassifier(n_estimators=10000, random_state=rs_, learning_rate=0.03,
                               n_jobs=-1,lambda_l1=1,feature_fraction=0.45,
                               scale_pos_weight=sc_pos_wt,
#                                max_depth=12,
#                                linear_tree=True,linear_lambda=0.1
                              )
        model.fit(train_x.loc[train_ids][feature_set], train_y.loc[train_ids], 
                  eval_set=[(train_x.loc[train_ids][feature_set], train_y.loc[train_ids]), 
                            (train_x.loc[test_ids][feature_set], train_y.loc[test_ids])],
                  verbose=False,
                  early_stopping_rounds=100,
                  categorical_feature=category_cols,
                  eval_metric=evaluate_F1_lgb
                 )
        pred_fold_i = model.predict_proba(train_x.loc[test_ids][feature_set])[:,-1]
        pred_fold.append(pred_fold_i)
        pred_fold_test_i = model.predict_proba(test_x[feature_set])[:,-1]
        pred_fold_test.append(pred_fold_test_i)
    pred_fold = [hmean(i) for i in [list(i) for i in zip(*pred_fold)]]
    pred_fold_test = [hmean(i) for i in [list(i) for i in zip(*pred_fold_test)]]
    
    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    pred_test.append([float(i) for i  in pred_fold_test])
    print("FOLD F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < f1_threshold, 0, 1))))
    df_train_error_analysis_fold = pd.DataFrame({'ID':list(train_x.loc[test_ids]['ID']),
                                        'ACTUAL_TARGET':list(train_y.loc[test_ids]),
                                        'PREDICTED_TARGET':pred_fold})
    df_train_error_analysis_lgb = pd.concat([df_train_error_analysis_lgb,df_train_error_analysis_fold],ignore_index=True)
    print('\n')

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_x.loc[train_ids][feature_set].columns
    fold_importance_df["importance"] = model.feature_importances_
    fold_importance_df["fold"] = j + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    j=j+1

# for i_thr in [np.round(i,2) for i in np.arange(0.2,0.6,0.01)]:
#     print("Threshold:",i_thr,"Final F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < i_thr, 0, 1))))
pred_x = list(np.where(np.array(pred_x) < f1_threshold, 0, 1))
print("Final F1 Score:", f1_score(pred_y, pred_x))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

# vote = final_prediction.mean(axis=1)
vote = final_prediction.apply(lambda x: hmean(x), axis=1)
final_prediction[id_variable] = test_id.values
final_prediction['data_rate_t+1_trend'] = vote
print(final_prediction.shape)

train_pred_all = pd.DataFrame()
train_pred_all['data_rate_t+1_trend'] = pred_y
train_pred_all[f'model_{version_nb}'] = pred_x
final_prediction_lgb = final_prediction.copy()

display(final_prediction.head())
df_submission = final_prediction[[id_variable,'data_rate_t+1_trend']].copy()
df_submission.to_csv(f"lgb_{version_nb}_raw.csv", index = False)
df_submission['data_rate_t+1_trend'] = list(np.where(final_prediction['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_submission.to_csv(f"lgb_{version_nb}.csv", index = False)
display(df_submission.head())

all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
all_features.reset_index(inplace=True)
important_features = list(all_features[all_features['importance']!=0]['feature'])
display(all_features)
display(all_features.head(25))

final_prediction_lgb = final_prediction[['ID','data_rate_t+1_trend']].copy()


● Fold : 1
[LightGBM] [Warning] feature_fraction is set=0.45, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
FOLD F1 Score: 0.7555555555555555


● Fold : 2
[LightGBM] [Warning] feature_fraction is set=0.45, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
FOLD F1 Score: 0.7410452143276571


● Fold : 3
[LightGBM] [Warning] feature_fraction is set=0.45, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
FOLD F1 Score: 0.7379844961240308


● Fold : 4
[LightGBM] [Warning] feature_fraction is set=0.45, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45
[LightGBM] [Warning] lambda_l1 is set=1, reg_al

,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4,FOLD_5,FOLD_6,FOLD_7,FOLD_8,FOLD_9,ID,data_rate_t+1_trend
0,0.574455,0.683709,0.674552,0.573004,0.645579,0.714688,0.609944,0.589983,0.677046,0.632587,B0017-25_24,0.634054
1,0.211372,0.171496,0.381589,0.081161,0.223958,0.143075,0.297911,0.308806,0.181911,0.155578,B0017-25_25,0.180251
2,0.234964,0.266496,0.304898,0.163412,0.244099,0.150218,0.383587,0.343519,0.177503,0.127916,B0017-25_26,0.212176
3,0.813275,0.767510,0.765483,0.848585,0.773569,0.831654,0.654581,0.668910,0.763209,0.855787,B0017-25_27,0.768410
4,0.646156,0.722562,0.771395,0.872548,0.726679,0.803796,0.595627,0.585576,0.836859,0.783163,B0017-32_1,0.721883


,ID,data_rate_t+1_trend
0,B0017-25_24,1
1,B0017-25_25,0
2,B0017-25_26,0
3,B0017-25_27,1
4,B0017-32_1,1


,feature,importance
0,data_rate_T-1,446.8
1,data_rate_T-72,180.2
2,hour,156.2
3,day,148.0
4,data_rate_T+1,134.5
...,...,...
113,endTime_T+1_hours_elapsed,1.6
114,endTime_T-3_hours_elapsed,1.1
115,endTime_T-2_hours_elapsed,1.0
116,endTime_T-1_hours_elapsed,0.2


,feature,importance
0,data_rate_T-1,446.8
1,data_rate_T-72,180.2
2,hour,156.2
3,day,148.0
4,data_rate_T+1,134.5
5,data_rate_T-24,124.0
6,data_rate_T-3,114.7
7,data_rate_T-48,113.3
8,data_rate_T-2,93.6
9,mcs_T-1,92.4


In [27]:
#0.7208
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
pred_test = []
pred_x = []
pred_y = []
j=0
df_train_error_analysis_cb = pd.DataFrame()
feature_importance_df = pd.DataFrame()
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('● Fold :', fold+1)
    pred_fold = []
    pred_fold_test = []
    for rs_ in seeds_rs:
        model = CatBoostClassifier(n_estimators=10000, random_state=rs_, learning_rate=0.03,
                                   eval_metric='F1:use_weights=False',
                                   scale_pos_weight=sc_pos_wt,
                                   use_best_model=True,
#                                    max_depth=8
                                  )
        model.fit(train_x.loc[train_ids][feature_set], train_y.loc[train_ids], 
                  eval_set=[(train_x.loc[train_ids][feature_set], train_y.loc[train_ids]), 
                            (train_x.loc[test_ids][feature_set], train_y.loc[test_ids])],
                  verbose=False,
                  early_stopping_rounds=100,
                  use_best_model=True,
                  cat_features=category_cols
    #               eval_metric=evaluate_F1_lgb
                 )
        pred_fold_i = model.predict_proba(train_x.loc[test_ids][feature_set])[:,-1]
        pred_fold.append(pred_fold_i)
        pred_fold_test_i = model.predict_proba(test_x[feature_set])[:,-1]
        pred_fold_test.append(pred_fold_test_i)
    pred_fold = [hmean(i) for i in [list(i) for i in zip(*pred_fold)]]
    pred_fold_test = [hmean(i) for i in [list(i) for i in zip(*pred_fold_test)]]
    

    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    
    pred_test.append([float(i) for i  in pred_fold_test])
    print("FOLD F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < f1_threshold, 0, 1))))
    df_train_error_analysis_fold = pd.DataFrame({'ID':list(train_x.loc[test_ids]['ID']),
                                        'ACTUAL_TARGET':list(train_y.loc[test_ids]),
                                        'PREDICTED_TARGET':pred_fold})
    df_train_error_analysis_cb = pd.concat([df_train_error_analysis_cb,df_train_error_analysis_fold],ignore_index=True)
    print('\n')

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_x.loc[train_ids][feature_set].columns
    fold_importance_df["importance"] = model.feature_importances_
    fold_importance_df["fold"] = j + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    j=j+1

# for i_thr in [np.round(i,2) for i in np.arange(0.2,0.6,0.01)]:
#     print("Threshold:",i_thr,"Final F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < i_thr, 0, 1))))
pred_x = list(np.where(np.array(pred_x) < f1_threshold, 0, 1))
print("Final F1 Score:", f1_score(pred_y, pred_x))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

# vote = final_prediction.mean(axis=1)
vote = final_prediction.apply(lambda x: hmean(x), axis=1)
final_prediction[id_variable] = test_id.values
final_prediction['data_rate_t+1_trend'] = vote
print(final_prediction.shape)

train_pred_all = pd.DataFrame()
train_pred_all['data_rate_t+1_trend'] = pred_y
train_pred_all[f'model_{version_nb}'] = pred_x
final_prediction_cb = final_prediction.copy()

display(final_prediction.head())
df_submission = final_prediction[[id_variable,'data_rate_t+1_trend']].copy()
df_submission.to_csv(f"cb_{version_nb}_raw.csv", index = False)
df_submission['data_rate_t+1_trend'] = list(np.where(final_prediction['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_submission.to_csv(f"cb_{version_nb}.csv", index = False)
display(df_submission.head())

all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
all_features.reset_index(inplace=True)
important_features = list(all_features[all_features['importance']!=0]['feature'])
display(all_features)
display(all_features.head(25))

final_prediction_cb = final_prediction[['ID','data_rate_t+1_trend']].copy()


● Fold : 1
FOLD F1 Score: 0.7364787111622555


● Fold : 2
FOLD F1 Score: 0.7192600652883571


● Fold : 3
FOLD F1 Score: 0.7210834553440703


● Fold : 4
FOLD F1 Score: 0.7224986180210061


● Fold : 5
FOLD F1 Score: 0.7237925662141109


● Fold : 6
FOLD F1 Score: 0.7273064785594952


● Fold : 7
FOLD F1 Score: 0.7244897959183675


● Fold : 8
FOLD F1 Score: 0.7244582043343653


● Fold : 9
FOLD F1 Score: 0.7229458917835672


● Fold : 10
FOLD F1 Score: 0.7208860051459894


Final F1 Score: 0.7208860051459894
(1932, 12)


,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4,FOLD_5,FOLD_6,FOLD_7,FOLD_8,FOLD_9,ID,data_rate_t+1_trend
0,0.629478,0.527471,0.608912,0.610433,0.660947,0.716370,0.612921,0.561749,0.666165,0.555428,B0017-25_24,0.610238
1,0.216149,0.436577,0.302705,0.273861,0.268643,0.228308,0.268842,0.291757,0.226159,0.455845,B0017-25_25,0.279822
2,0.241329,0.480752,0.238471,0.304235,0.261770,0.294681,0.299345,0.183562,0.197867,0.428870,B0017-25_26,0.269602
3,0.794626,0.567015,0.740273,0.719900,0.724300,0.761737,0.645518,0.775641,0.799794,0.621773,B0017-25_27,0.706591
4,0.804393,0.546573,0.758041,0.576334,0.697615,0.709021,0.663143,0.810404,0.854319,0.522916,B0017-32_1,0.675872


,ID,data_rate_t+1_trend
0,B0017-25_24,1
1,B0017-25_25,0
2,B0017-25_26,0
3,B0017-25_27,1
4,B0017-32_1,1


,feature,importance
0,data_rate_T-1,23.715811
1,bler_T-1,4.605207
2,TA_T-1,3.993028
3,hour,3.925699
4,cqi_T-1,3.421822
...,...,...
113,endTime_T-2_hours_elapsed,0.057594
114,endTime_T-1_hours_elapsed,0.046916
115,endTime_T-3_hours_elapsed,0.043809
116,endTime_T+2_hours_elapsed,0.038140


,feature,importance
0,data_rate_T-1,23.715811
1,bler_T-1,4.605207
2,TA_T-1,3.993028
3,hour,3.925699
4,cqi_T-1,3.421822
5,data_rate_T+1,3.254145
6,hour_cos,3.025102
7,mcs_T-1,2.612342
8,data_rate_T-72,2.411018
9,data_rate_T-48,2.331079


In [28]:
#0.7180
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
pred_test = []
pred_x = []
pred_y = []
j=0
df_train_error_analysis_hgb = pd.DataFrame()
feature_importance_df = pd.DataFrame()
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('● Fold :', fold+1)
    pred_fold = []
    pred_fold_test = []
    for rs_ in seeds_rs:
        model = HistGradientBoostingClassifier(random_state=rs_,verbose=0,
                                               scoring = 'recall',
                                              early_stopping = True,
                                               max_iter = 10000,
                                              learning_rate = 0.03,
                                              n_iter_no_change = 100,
                                               class_weight={1:sc_pos_wt},
                                               categorical_features=category_cols
                                               )
        model.fit(train_x.loc[train_ids][feature_set], train_y.loc[train_ids])
        pred_fold_i = model.predict_proba(train_x.loc[test_ids][feature_set])[:,-1]
        pred_fold.append(pred_fold_i)
        pred_fold_test_i = model.predict_proba(test_x[feature_set])[:,-1]
        pred_fold_test.append(pred_fold_test_i)
    pred_fold = [hmean(i) for i in [list(i) for i in zip(*pred_fold)]]
    pred_fold_test = [hmean(i) for i in [list(i) for i in zip(*pred_fold_test)]]
    
    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    pred_test.append([float(i) for i  in pred_fold_test])
    print("FOLD F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < f1_threshold, 0, 1))))
    df_train_error_analysis_fold = pd.DataFrame({'ID':list(train_x.loc[test_ids]['ID']),
                                        'ACTUAL_TARGET':list(train_y.loc[test_ids]),
                                        'PREDICTED_TARGET':pred_fold})
    df_train_error_analysis_hgb = pd.concat([df_train_error_analysis_hgb,df_train_error_analysis_fold],ignore_index=True)
    print('\n')

#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["feature"] = train_x.loc[train_ids][feature_set].columns
#     fold_importance_df["importance"] = model.feat
#     fold_importance_df["fold"] = j + 1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
#     j=j+1

# for i_thr in [np.round(i,2) for i in np.arange(0.2,0.6,0.01)]:
#     print("Threshold:",i_thr,"Final F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < i_thr, 0, 1))))
pred_x = list(np.where(np.array(pred_x) < f1_threshold, 0, 1))
print("Final F1 Score:", f1_score(pred_y, pred_x))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

# vote = final_prediction.mean(axis=1)
vote = final_prediction.apply(lambda x: hmean(x), axis=1)
final_prediction[id_variable] = test_id.values
final_prediction['data_rate_t+1_trend'] = vote
print(final_prediction.shape)

train_pred_all = pd.DataFrame()
train_pred_all['data_rate_t+1_trend'] = pred_y
train_pred_all[f'model_{version_nb}'] = pred_x
final_prediction_hgb = final_prediction.copy()

display(final_prediction.head())
df_submission = final_prediction[[id_variable,'data_rate_t+1_trend']].copy()
df_submission.to_csv(f"hgb_{version_nb}_raw.csv", index = False)
df_submission['data_rate_t+1_trend'] = list(np.where(final_prediction['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_submission.to_csv(f"hgb_{version_nb}.csv", index = False)
display(df_submission.head())

# all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
# all_features.reset_index(inplace=True)
# important_features = list(all_features[all_features['importance']!=0]['feature'])
# display(all_features)
# display(all_features.head(25))

final_prediction_hgb = final_prediction[['ID','data_rate_t+1_trend']].copy()


● Fold : 1
FOLD F1 Score: 0.7259953161592507


● Fold : 2
FOLD F1 Score: 0.7169811320754716


● Fold : 3
FOLD F1 Score: 0.7198428290766208


● Fold : 4
FOLD F1 Score: 0.7160714285714287


● Fold : 5
FOLD F1 Score: 0.7178383548541368


● Fold : 6
FOLD F1 Score: 0.7214285714285714


● Fold : 7
FOLD F1 Score: 0.7204355963927174


● Fold : 8
FOLD F1 Score: 0.7193687658180735


● Fold : 9
FOLD F1 Score: 0.7185770750988142


● Fold : 10
FOLD F1 Score: 0.7180945609669392


Final F1 Score: 0.7180945609669392
(1932, 12)


,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4,FOLD_5,FOLD_6,FOLD_7,FOLD_8,FOLD_9,ID,data_rate_t+1_trend
0,0.519403,0.651610,0.719223,0.635364,0.659259,0.701838,0.629464,0.693526,0.587924,0.593698,B0017-25_24,0.633643
1,0.314811,0.219840,0.263908,0.200861,0.277674,0.231827,0.181603,0.272432,0.206644,0.205728,B0017-25_25,0.231026
2,0.408433,0.282503,0.239225,0.241778,0.207713,0.251335,0.223651,0.253038,0.240618,0.270990,B0017-25_26,0.254021
3,0.786613,0.753444,0.786421,0.832658,0.756345,0.732322,0.736342,0.787052,0.754270,0.808685,B0017-25_27,0.772217
4,0.604408,0.674357,0.685888,0.650540,0.652066,0.724493,0.598357,0.576309,0.726376,0.585677,B0017-32_1,0.643626


,ID,data_rate_t+1_trend
0,B0017-25_24,1
1,B0017-25_25,0
2,B0017-25_26,0
3,B0017-25_27,1
4,B0017-32_1,1


In [29]:
# train_x[category_cols] = train_x[category_cols].astype("category")
# test_x[category_cols] = test_x[category_cols].astype("category")
#  enable_categorical=True,

In [30]:
# 0.7290
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
pred_test = []
pred_x = []
pred_y = []
j=0
df_train_error_analysis_xgb = pd.DataFrame()
feature_importance_df = pd.DataFrame()
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('● Fold :', fold+1)
    pred_fold = []
    pred_fold_test = []
    for rs_ in seeds_rs:
        model = XGBClassifier(n_estimators=10000, random_state=rs_, learning_rate=0.03,
                              n_jobs=-1,scale_pos_weight=sc_pos_wt,colsample_bytree=0.45,
                              reg_lambda=1,reg_alpha=0,max_depth=6,
                             disable_default_eval_metric=1,
#                               tree_method='exact',
                              tree_method='hist',
                              enable_categorical=True
                             )
        model.fit(train_x.loc[train_ids][feature_set], train_y.loc[train_ids], 
                  eval_set=[(train_x.loc[train_ids][feature_set], train_y.loc[train_ids]), 
                            (train_x.loc[test_ids][feature_set], train_y.loc[test_ids])],
                  verbose=False,
                  early_stopping_rounds=100,
                  eval_metric=evaluate_F1_xgb
                 )
        pred_fold_i = model.predict_proba(train_x.loc[test_ids][feature_set])[:,-1]
        pred_fold.append(pred_fold_i)
        pred_fold_test_i = model.predict_proba(test_x[feature_set])[:,-1]
        pred_fold_test.append(pred_fold_test_i)
    pred_fold = [hmean(i) for i in [list(i) for i in zip(*pred_fold)]]
    pred_fold_test = [hmean(i) for i in [list(i) for i in zip(*pred_fold_test)]]
    
    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    
    pred_test.append([float(i) for i  in pred_fold_test])
    print("FOLD F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < f1_threshold, 0, 1))))
    df_train_error_analysis_fold = pd.DataFrame({'ID':list(train_x.loc[test_ids]['ID']),
                                        'ACTUAL_TARGET':list(train_y.loc[test_ids]),
                                        'PREDICTED_TARGET':pred_fold})
    df_train_error_analysis_xgb = pd.concat([df_train_error_analysis_xgb,df_train_error_analysis_fold],ignore_index=True)
    print('\n')

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_x.loc[train_ids][feature_set].columns
    fold_importance_df["importance"] = model.feature_importances_
    fold_importance_df["fold"] = j + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    j=j+1

# for i_thr in [np.round(i,2) for i in np.arange(0.2,0.6,0.01)]:
#     print("Threshold:",i_thr,"Final F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < i_thr, 0, 1))))
pred_x = list(np.where(np.array(pred_x) < f1_threshold, 0, 1))
print("Final F1 Score:", f1_score(pred_y, pred_x))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

# vote = final_prediction.mean(axis=1)
vote = final_prediction.apply(lambda x: hmean(x), axis=1)
final_prediction[id_variable] = test_id.values
final_prediction['data_rate_t+1_trend'] = vote
print(final_prediction.shape)

train_pred_all = pd.DataFrame()
train_pred_all['data_rate_t+1_trend'] = pred_y
train_pred_all[f'model_{version_nb}'] = pred_x
final_prediction_xgb = final_prediction.copy()

display(final_prediction.head())
df_submission = final_prediction[[id_variable,'data_rate_t+1_trend']].copy()
df_submission.to_csv(f"xgb_{version_nb}_raw.csv", index = False)
df_submission['data_rate_t+1_trend'] = list(np.where(final_prediction['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_submission.to_csv(f"xgb_{version_nb}.csv", index = False)
display(df_submission.head())

all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
all_features.reset_index(inplace=True)
important_features = list(all_features[all_features['importance']!=0]['feature'])
display(all_features)
display(all_features.head(25))

final_prediction_xgb = final_prediction[['ID','data_rate_t+1_trend']].copy()



● Fold : 1
FOLD F1 Score: 0.7412587412587412


● Fold : 2
FOLD F1 Score: 0.7335600907029478


● Fold : 3
FOLD F1 Score: 0.7319311663479924


● Fold : 4
FOLD F1 Score: 0.7292025243832472


● Fold : 5
FOLD F1 Score: 0.7276478679504814


● Fold : 6
FOLD F1 Score: 0.7315397824842588


● Fold : 7
FOLD F1 Score: 0.7300863332790357


● Fold : 8
FOLD F1 Score: 0.7297723041672634


● Fold : 9
FOLD F1 Score: 0.7281059063136457


● Fold : 10
FOLD F1 Score: 0.7290151252742176


Final F1 Score: 0.7290151252742176
(1932, 12)


,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4,FOLD_5,FOLD_6,FOLD_7,FOLD_8,FOLD_9,ID,data_rate_t+1_trend
0,0.493870,0.658537,0.555510,0.547297,0.697966,0.586659,0.626242,0.616787,0.677395,0.542319,B0017-25_24,0.593604
1,0.178900,0.255623,0.205681,0.196560,0.254430,0.237663,0.322688,0.230859,0.148287,0.115168,B0017-25_25,0.198230
2,0.267464,0.327355,0.222791,0.291761,0.268865,0.360049,0.288019,0.225462,0.168941,0.084803,B0017-25_26,0.213561
3,0.779241,0.684872,0.699530,0.714055,0.628247,0.631934,0.621539,0.730515,0.693631,0.828554,B0017-25_27,0.695700
4,0.696113,0.729666,0.864184,0.728782,0.709176,0.757725,0.662465,0.734196,0.842481,0.787381,B0017-32_1,0.746562


,ID,data_rate_t+1_trend
0,B0017-25_24,1
1,B0017-25_25,0
2,B0017-25_26,0
3,B0017-25_27,1
4,B0017-32_1,1


,feature,importance
0,hour_cos,0.026087
1,data_rate_T-1,0.025518
2,cqi_T-1,0.020328
3,data_rate_T+1,0.019436
4,data_rate_T-48,0.015973
...,...,...
113,fault_duration_T-24,0.005763
114,mcs_T+72,0.005503
115,endTime_T-2_hours_elapsed,0.005482
116,endTime_T-3_hours_elapsed,0.003968


,feature,importance
0,hour_cos,0.026087
1,data_rate_T-1,0.025518
2,cqi_T-1,0.020328
3,data_rate_T+1,0.019436
4,data_rate_T-48,0.015973
5,endTime_T+2_hours_elapsed,0.015244
6,hour,0.015187
7,mcs_T-1,0.015104
8,data_rate_T-72,0.014351
9,hour_sin,0.013840


In [31]:
for df in [final_prediction_lgb,final_prediction_cb,final_prediction_hgb,final_prediction_xgb]:
    print(df['data_rate_t+1_trend'].min(),df['data_rate_t+1_trend'].max())

0.04757672435785618 0.8915715933409171
0.047671685868941244 0.85507684254002
0.03211506264784815 0.8968380571303354
0.06319127033899936 0.8909012934326843


In [32]:
#0.7234
df_local_cv = df_train_error_analysis_lgb.set_index(["ID","ACTUAL_TARGET"]).add_suffix("_LGB").join(df_train_error_analysis_xgb.set_index(["ID","ACTUAL_TARGET"]).add_suffix("_XGB")).join(df_train_error_analysis_cb.set_index(["ID","ACTUAL_TARGET"]).add_suffix("_CB")).join(df_train_error_analysis_hgb.set_index(["ID","ACTUAL_TARGET"]).add_suffix("_HGB"))
# vote = df_local_cv['PREDICTED_TARGET_LGB']*0.4+df_local_cv['PREDICTED_TARGET_XGB']*0.4+df_local_cv['PREDICTED_TARGET_HGB']*0.1+df_local_cv['PREDICTED_TARGET_CB']*0.1
vote = df_local_cv.apply(lambda x: hmean(x), axis=1)
df_local_cv['data_rate_t+1_trend'] = vote
df_local_cv['data_rate_t+1_trend'] = list(np.where(df_local_cv['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_local_cv = df_local_cv.reset_index()
f1_score(df_local_cv['ACTUAL_TARGET'],df_local_cv['data_rate_t+1_trend'])

0.723409223584355

In [33]:
df_ensemble = final_prediction_lgb.set_index("ID").add_suffix("_LGB").join(final_prediction_cb.set_index("ID").add_suffix("_CB")).join(final_prediction_hgb.set_index("ID").add_suffix("_HGB")).join(final_prediction_xgb.set_index("ID").add_suffix("_XGB"))
display(df_ensemble.corr())
vote = df_ensemble.apply(lambda x: hmean(x), axis=1)
# vote = df_ensemble['data_rate_t+1_trend_LGB']*0.4+df_ensemble['data_rate_t+1_trend_XGB']*0.4+df_ensemble['data_rate_t+1_trend_HGB']*0.1+df_ensemble['data_rate_t+1_trend_CB']*0.1
df_ensemble['data_rate_t+1_trend'] = vote
df_ensemble = df_ensemble[['data_rate_t+1_trend']].copy()
df_ensemble_raw = df_ensemble.reset_index()
df_ensemble_raw.to_csv(f"ensemble_all_{version_nb}_{sc_pos_wt}_raw.csv", index = False)
df_ensemble['data_rate_t+1_trend'] = list(np.where(df_ensemble['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_ensemble = df_ensemble.reset_index()
df_ensemble.to_csv(f"ensemble_all_{version_nb}_{sc_pos_wt}.csv", index = False)


,data_rate_t+1_trend_LGB,data_rate_t+1_trend_CB,data_rate_t+1_trend_HGB,data_rate_t+1_trend_XGB
data_rate_t+1_trend_LGB,1.000000,0.939039,0.975761,0.928104
data_rate_t+1_trend_CB,0.939039,1.000000,0.923429,0.914547
data_rate_t+1_trend_HGB,0.975761,0.923429,1.000000,0.902297
data_rate_t+1_trend_XGB,0.928104,0.914547,0.902297,1.000000


In [34]:
df_ensemble['data_rate_t+1_trend'].value_counts(normalize=True)

1    0.70911
0    0.29089
Name: data_rate_t+1_trend, dtype: float64